In [8]:
# !pip3 install openpyxl

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 242 kB 1.3 MB/s eta 0:00:01
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [1]:
import requests
import base64
import os
import numpy as np
import pandas as pd
import json
import time

In [2]:
class Spotify():
    '''
    Class to handle hitting spotify API to get music features
    client_id = encrypted spotify client ID for data_engineering@soul-cycle
    client_secret = encrypted spotify client secret
    '''
    
    def __init__(self, 
                 client_id=None, 
                 client_secret=None):
        
        self.client_id = client_id
        self.client_secret = client_secret
        
        self.get_access_token()
        
        
    def get_access_token(self):
        ## Example use:
        ## spotify_instance = Spotify(CLIENT_ID, CLIENT_SECRET)
        ## spotify_instance.get_access_token()
        
        secret_bytes = bytes(('{}:{}'.format(self.client_id, self.client_secret)),'utf-8')
        secret_enc = base64.b64encode(secret_bytes).decode('utf-8')
        
        data = {'grant_type': 'client_credentials'}
        headers = {'Authorization': 'Basic {}'.format(secret_enc)}
        url = 'https://accounts.spotify.com/api/token'
        r = requests.post(url, headers=headers, data=data)
        
        self.access_token = r.json()['access_token']
    
    def group_ids(self, list_of_ids, bucket_size=100):
        list_of_ids = pd.DataFrame(list_of_ids, columns=['id'])
        list_of_ids['bucket'] = np.floor(np.arange(len(list_of_ids)) / bucket_size)
        
        gpd_ids = list_of_ids.groupby('bucket')['id'].apply(lambda x: ','.join(x))
        
        return gpd_ids

    def get_audio_features(self, id_list, bucket_size=100):
        
        gpd_ids = self.group_ids(id_list, bucket_size)
        col_list = ['acousticness', 'liveness', 'instrumentalness', 
                    'analysis_url', 'uri', 'time_signature', 'loudness', 
                    'speechiness', 'duration_ms', 'danceability', 'mode', 
                    'id', 'energy', 'key', 'track_href', 
                    'valence', 'type', 'tempo']
        
        df = []
        
        for x in gpd_ids.values:
            r = requests.get('https://api.spotify.com/v1/audio-features?ids={}'.format(x), headers = 
                                {'Authorization': 'Bearer ' + self.access_token})
            while r.status_code == 429:
                retry_secs = int(r.headers['Retry-After'])
                time.sleep(retry_secs)
            
            audio_features = r.json()['audio_features']
            audio_features = [i for i in audio_features if i]
            
            for row in audio_features:
                for c in col_list:
                    if c not in row:
                        row[c] = None
                df.append(row)
    
        return pd.DataFrame(df)
    
    def get_tracks(self, id_list, bucket_size=50):
        
        gpd_ids = self.group_ids(id_list, bucket_size)
        col_list = ['popularity', 'name', 'uri', 'external_urls', 'type', 'duration_ms', 'external_ids',
                    'album', 'explicit', 'id', 'preview_url', 'track_number', 'available_markets', 'is_local', 
                    'artists', 'href', 'disc_number']
        
        track_df = []
        
        for x in gpd_ids.values:
            r = requests.get('https://api.spotify.com/v1/tracks?ids={}'.format(x), headers = 
                             {'Authorization': 'Bearer ' + self.access_token})
            
            while r.status_code == 429:
                retry_secs = int(r.headers['Retry-After'])
                time.sleep(retry_secs)
                  
            tracks = r.json()['tracks']
            tracks = [i for i in tracks if i]
            
            for row in tracks:
                for c in col_list:
                    if c not in row:
                        row[c] = None
                track_df.append(row)
        
        final_track_df = pd.DataFrame(track_df)
        
        final_track_df['release_date'] = pd.json_normalize(final_track_df['album'])['release_date']
        final_track_df['album_id'] = pd.json_normalize(final_track_df['album'])['id']
        
        return final_track_df[['id','popularity','album_id','release_date']]

    def get_albums(self, id_list, bucket_size=20):
        
        gpd_ids = self.group_ids(id_list, bucket_size)
        col_list = ['id', 'copyright']
        
        album_df = []
        
        for x in gpd_ids.values:
            r = requests.get('https://api.spotify.com/v1/albums?ids={}'.format(x), headers = 
                             {'Authorization': 'Bearer ' + self.access_token})
            
            while r.status_code == 429:
                retry_secs = int(r.headers['Retry-After'])
                time.sleep(retry_secs)
                  
            albums = r.json()['albums']
            albums = [i for i in albums if i]
            
            for row in albums:
                for c in col_list:
                    if c not in row:
                        row[c] = None
                album_df.append(row)
                
        final_album_df = pd.DataFrame(album_df).explode('copyrights')[['copyrights','id']]
        return final_album_df

In [3]:
client_id = 'ae17fb14354d4d98a442007563fafab9'
client_secret = 'cb9e3752f07f4cfc9685c9369fd4b11c'

In [4]:
sp = Spotify(client_id,client_secret)

In [5]:
sp.get_access_token()

In [15]:
music_data = pd.read_excel('mri_data.xlsx', index=False)

In [13]:
music_data.drop('Unnamed: 0', axis=1, inplace=True)

In [14]:
music_data.head()

,Statement Month,Store Name,Artist Name,Release Name,Track Artist (Performer),Track Name,ISRC,Display Upc,Country Code,Continent,"Stream Ad-Supp, Stream Prem, Download, Physical, Locker, Other",Units,Gross Revenue USD
0,2021-03,Pandora,Rebelution,Count Me In,Rebelution,Roots Reggae Music (feat. Don Carlos),US4CL1410011,886444576840,US,North America,Stream Premium,280323,3373.37
1,2021-03,Pandora,Rebelution,Count Me In,Rebelution,Roots Reggae Music (feat. Don Carlos),US4CL1410011,886444576840,US,North America,Stream Ad-Supported,635968,3223.90
2,2021-02,Pandora,Rebelution,Count Me In,Rebelution,Roots Reggae Music (feat. Don Carlos),US4CL1410011,886444576840,US,North America,Stream Premium,282556,3063.66
3,2021-01,Pandora,Rebelution,Count Me In,Rebelution,Roots Reggae Music (feat. Don Carlos),US4CL1410011,886444576840,US,North America,Stream Premium,252873,2970.30
4,2021-03,Spotify,Rebelution,Count Me In,Rebelution,Fade Away,US4CL1410008,886444576840,US,North America,Stream Premium,479870,2240.75


In [16]:
len(music_data.ISRC.unique())

1062

In [17]:
len(music_data)

369017